In [ ]:
%%bash
pip install pandas
pip install git+https://github.com/openai/whisper.git
pip install pytube
pip install numpy
pip install pinecone-client

In [ ]:
# @title
import os
import torch
import whisper
import pinecone
import numpy as np
import pandas as pd
from pytube import YouTube

In [ ]:
def video_to_audio(video_url, destination):
    # Get the video
    video = YouTube(video_url)
    # Convert video to Audio
    audio = video.streams.filter(only_audio=True).first()
    # Save to destination
    output = audio.download(output_path = destination)
    name, ext = os.path.splitext(output)
    new_file = name + '.mp3'
    # Replace spaces with "_"
    new_file = new_file.replace(" ", "_")
    # Change the name of the file
    os.rename(output, new_file)
    return new_file

In [ ]:
!mkdir audio_data

mkdir: cannot create directory ‘audio_data’: File exists


In [ ]:
# Create URL column
audio_path = "audio_data"

In [ ]:
# truncated list of links
list_videos = ["https://www.youtube.com/watch?v=CXiAoaMPXdU"]
# Create dataframe
transcription_df = pd.DataFrame(list_videos, columns=['URLs'])
transcription_df.head()

,URLs
0,https://www.youtube.com/watch?v=CXiAoaMPXdU


In [ ]:
transcription_df["file_name"] = transcription_df["URLs"].apply(
    lambda url: video_to_audio(url, audio_path)
)
transcription_df.head()

,URLs,file_name
0,https://www.youtube.com/watch?v=CXiAoaMPXdU,/content/audio_data/Kaj_je_metoda_bisekcije.mp3


In [ ]:
!nvidia-smi

Fri Feb 16 22:01:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load the model
whisper_model = whisper.load_model("large", device=device)

100%|██████████████████████████████████████| 2.88G/2.88G [00:29<00:00, 105MiB/s]


In [ ]:
def audio_to_text(audio_file):
   return whisper_model.transcribe(audio_file)["text"]

In [ ]:
# Apply the function to all the audio files
transcription_df["transcriptions"] = transcription_df["file_name"].apply(lambda f_name: audio_to_text(f_name))

In [ ]:
# Show the first five rows
transcription_df.head()

,URLs,file_name,transcriptions
0,https://www.youtube.com/watch?v=CXiAoaMPXdU,/content/audio_data/Kaj_je_metoda_bisekcije.mp3,V tem kratkem poglavju si bomo pogledali post...
